### Ejercicio GroupBy N1

In [0]:
%run "../includes/configuration"

#### Obtener los DF

In [0]:
movie_df = spark.read.parquet(f"{silver_foler_path}/movies")
genre_df = spark.read.parquet(f"{silver_foler_path}/genre")
movie_genre_df = spark.read.parquet(f"{silver_foler_path}/movie_genre")

#### Cruzar los DF

In [0]:
display(movie_df)

In [0]:
movie_genre_final_df = movie_df.join(movie_genre_df,movie_df.movie_id == movie_genre_df.movie_id,"inner") \
                                .join(genre_df, genre_df.genre_id == movie_genre_df.genre_id,"inner") \
                                .where(movie_df.year_release_date >= 2015) \
                                .select(movie_df.movie_id,movie_df.budget,movie_df.release_date,
                                        movie_df.revenue,genre_df.genre_name, movie_df.year_release_date)

In [0]:
display(movie_genre_final_df)

#### Realizar las agrupaciones

In [0]:
from pyspark.sql.functions import sum, min, max, dense_rank, desc
from pyspark.sql.window import Window

#Agrupar por año de lanzamiento y genero
movie_genre_group_df =  movie_genre_final_df.groupBy("year_release_date","genre_name") \
                    .agg(
                      sum("budget").alias("total_presupuesto"),
                      sum("revenue").alias("total_ingresos"),
                    )

In [0]:
#Realizar ranking de manera acendente por el total_presupuesto y total_ingresos particionado por el año de lanzamiento

movie_genre_group_df.withColumn("rank",dense_rank().over(Window.partitionBy("year_release_date")
                                                         .orderBy(desc("total_presupuesto"),desc("total_ingresos")))).display()

#### Escribir el el DataLake de GOLD

In [0]:
movie_genre_group_df.write.mode("overwrite").parquet(f"{gold_folder_path}/results_group_movie_genre")

In [0]:
display(spark.read.parquet(f"{gold_folder_path}/results_group_movie_genre"))